
# **Interactive Variogram Calculation and Modeling Demonstration**

## **Reidar B. Bratvold, Professor, University of Stavanger**  

## **Overview**
This interactive workflow demonstrates the calculation of **directional experimental variograms** in 2D. It is essential for quantifying spatial continuity in **sparsely sampled, irregular spatial data**.

For a more comprehensive approach to variogram calculation, consider these resources:
- [Experimental Variogram Calculation in Python with GeostatsPy](https://github.com/GeostatsGuy/PythonNumericalDemos/blob/master/GeostatsPy_variogram_calculation.ipynb)
- [Determination of Major and Minor Spatial Continuity Directions in Python with GeostatsPy](https://github.com/GeostatsGuy/PythonNumericalDemos/blob/master/GeostatsPy_spatial_continuity_directions.ipynb)

---

## **Spatial Continuity**
**Spatial continuity** refers to the correlation of values over distance.

- **No spatial continuity** → No correlation between values, meaning data points are completely random.
- **Perfect spatial continuity** → Homogeneous phenomena where all values are highly correlated.

To quantify spatial continuity, we use the **semivariogram**.

---

## **The Semivariogram**
The **semivariogram** quantifies spatial variability as a function of separation distance:

$$
\gamma(\mathbf{h}) = \frac{1}{2 N(\mathbf{h})} \sum^{N(\mathbf{h})}_{\alpha=1} (z(\mathbf{u}_\alpha) - z(\mathbf{u}_\alpha + \mathbf{h}))^2  
$$

where:
- \( z(\mathbf{u}_\alpha) \) and \( z(\mathbf{u}_\alpha + \mathbf{h}) \) are sample values at the **tail** and **head** of the lag vector \( \mathbf{h} \).
- \( N(\mathbf{h}) \) is the number of paired data points at that lag.

The **semivariogram** is computed over multiple lag distances to obtain a **continuous function**.

### **Relationship to Covariance and Correlogram**
The semivariogram relates to the **covariance function** \( C_x(\mathbf{h}) \) and **variance** \( \sigma^2_x \):

$$
C_x(\mathbf{h}) = \sigma^2_x - \gamma(\mathbf{h})
$$

The **correlogram** measures correlation as:

$$
\rho_x(\mathbf{h}) = \frac{C_x(\mathbf{h})}{\sigma^2_x}
$$

which satisfies:

$$
-1.0 \leq \rho_x(\mathbf{h}) \leq 1.0
$$

---

## **Variogram Observations**
### **Observation #1: Variability Generally Increases with Distance**
- As lag distance increases, spatial correlation typically decreases.
- Some exceptions include **cyclic structures** (e.g., hole-effect models) where the variogram decreases over certain lag distances.

### **Observation #2: Variograms Use All Possible Pairs Separated by \( \mathbf{h} \)**
- The semivariogram is computed from **all possible data pairs** within a given lag distance.
- More pairs result in **more stable** variogram estimates.

### **Observation #3: The Sill Represents the Degree of Correlation**
- The **sill** is the total variance \( \sigma^2_x \).
- The covariance function follows:

  $$
  C_x(\mathbf{h}) = \sigma^2_x - \gamma(\mathbf{h})
  $$

- If the data variance is **standardized to 1.0**, then:

  $$
  \rho_x(\mathbf{h}) = \sigma^2_x - \gamma(\mathbf{h})
  $$

### **Observation #4: The Range Defines the Limit of Correlation**
- The **range** is the lag distance where the variogram reaches the sill.
- Beyond the **range**, knowing a sample provides **no information** about another.

### **Observation #5: The Nugget Effect (Discontinuity at \( h = 0 \))**
- A sudden jump at **small lags** is called the **nugget effect**.
- Causes include:
  - **Measurement error**
  - **Small-scale variability**
  - **Mixed populations**

The **relative nugget effect** is defined as:

$$
\frac{\text{nugget}}{\text{sill}} \times 100\%
$$

Caution: A nugget effect can **mask spatial structure** if misinterpreted.

---

## **Variogram Calculation Parameters**
### **Key Parameters**
- **Azimuth (\( \theta \))**: Direction of the lag vector.
- **Azimuth tolerance**: Maximum deviation from the azimuth (set to 90° for isotropic variograms).
- **Unit lag distance**: Bin size for lag distances (typically set to the minimum data spacing).
- **Lag distance tolerance**: Acceptable variation in lag distance (commonly 50% of unit lag).
- **Number of lags**: Typically **limited to half the dataset extent** to avoid unreliable estimates.
- **Bandwidth**: Maximum allowable deviation from the lag vector.

---

## **Variogram Modeling**
To model spatial continuity, we use **nested variogram structures**:

$$
\Gamma_x(\mathbf{h}) = \sum_{i=1}^{n_{\text{st}}} \gamma_i(\mathbf{h})
$$

where:
- \( \Gamma_x(\mathbf{h}) \) is the modeled variogram.
- \( n_{\text{st}} \) is the number of **nested** variogram structures.

### **Common Variogram Models**
1. **Spherical**
2. **Exponential**
3. **Gaussian**
4. **Nugget**

### **Less Common Models**
- **Hole-effect**
- **Damped hole-effect**
- **Power-law** (for fractal-like data)

Each variogram component follows a **geometric anisotropy model**:

$$
\mathbf{h}_i = \sqrt{\left(\frac{r_{\text{maj}}}{a_{\text{maj},i}}\right)^2 + \left(\frac{r_{\text{min}}}{a_{\text{min},i}}\right)^2}
$$

where:
- \( r_{\text{maj}}, r_{\text{min}} \) are distances in the **major** and **minor** directions.
- \( a_{\text{maj}}, a_{\text{min}} \) are the **ranges** in these directions.

---

## **Getting Started**
Before running the workflow, ensure you have the required **sample dataset**:

📂 **Data File:** `sample_data.csv`



### Import libraries


In [ ]:
%matplotlib inline
import geostats    
import os                                               # to set current working directory 
import sys                                              # supress output to screen for interactive variogram modeling
import io
import numpy as np                                      # arrays and matrix math
import pandas as pd                                     # DataFrames
import matplotlib.pyplot as plt                         # plotting
from matplotlib.pyplot import cm                        # color maps
from matplotlib import patches                          # draw variogram ellipse
from ipywidgets import interactive                      # widgets and interactivity
from ipywidgets import widgets                            
from ipywidgets import Layout
from ipywidgets import Label
from ipywidgets import VBox, HBox
import math
from math import sin, cos, radians, pi

import warnings
warnings.filterwarnings("ignore")

## We need to define three key function that will be used in the calculations

- `vmodel_struct()`: Computes **variogram model values** (semivariance, covariance, correlation) for a given lag distance & azimuth.

- `cova2_struct()`: Computes **covariance** for a given lag distance based on the selected **variogram model** (spherical, exponential, Gaussian, power-law).

- `point_pos()`: Computes **spatial coordinates** of a point given an **initial position, distance, and azimuth**.

## **1. `vmodel_struct(nlag, xlag, azm, vario, istruct)`**

### **Purpose**
This function computes **variogram model values** (semivariance, covariance, and correlation) for a given **lag distance** and azimuthal direction.

### **Inputs**
- `nlag`: Number of lag distances to compute.
- `xlag`: Lag distance increment.
- `azm`: Azimuth direction (in degrees).
- `vario`: Dictionary containing variogram model parameters.
- `istruct`: Structure index (determines which variogram component to compute).

### **Process**
1. **Initialize Constants & Variables**:
   - Define `MAXNST` (maximum structures) and `DEG2RAD` (degree-to-radian conversion).
   - Initialize arrays for **index, lag distances, semivariance (γ(h)), covariance, and correlation**.

2. **Load Variogram Parameters**:
    - Reads **nugget effect (`c0`) and nested structures (`cc`, `it`, `ang`, `aa`, `anis`)**.
    - Supports **two nested variogram structures**.

3. **Compute Spatial Offsets**:
   - Converts **azimuth to x and y offsets**.

4. **Compute Variogram Values**:
   - Loops over all lag distances:
     - Computes **covariance using `cova2_struct()`**.
     - Computes **semivariance**:
       $$ \gamma(h) = 	ext{maxcov} - 	ext{cov}(h) $$
     - Computes **correlation**:
       $$ 
ho(h) = rac{	ext{cov}(h)}{	ext{maxcov}} $$
     - Updates **spatial coordinates** (`xx, yy`).

5. **Returns**:
   - `index`: Lag indices.
   - `h`: Lag distances.
   - `gam`: Semivariance values.
   - `cov`: Covariance values.
   - `ro`: Correlation values.

In [ ]:
import math

def vmodel_struct(nlag: int, xlag: float, azm: float, vario: dict, istruct: int):
    """
    Computes the variogram model values including semivariance, covariance, 
    and correlation for a given lag distance and azimuth.

    Parameters:
    -----------
    nlag : int
        Number of lag distances to compute.
    xlag : float
        Lag distance increment.
    azm : float
        Azimuth direction in degrees.
    vario : dict
        Dictionary containing variogram model parameters.
    istruct : int
        Structure index for selecting the variogram component.

    Returns:
    --------
    tuple (np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray)
        - index: Lag indices.
        - h: Lag distances.
        - gam: Semivariance values.
        - cov: Covariance values.
        - ro: Correlation values.
    """

    # Constants
    DEG2RAD = math.pi / 180.0  

    # Initialize output arrays
    index = np.zeros(nlag + 1)
    h = np.zeros(nlag + 1)
    gam = np.zeros(nlag + 1)
    cov = np.zeros(nlag + 1)
    ro = np.zeros(nlag + 1)

    # Load variogram parameters from dictionary
    nst = vario["nst"]
    c0 = vario["nug"]

    # Allocate structure arrays
    cc = np.zeros(nst)
    aa = np.zeros(nst)
    it = np.zeros(nst)
    ang = np.zeros(nst)
    anis = np.zeros(nst)

    # Assign primary structure
    cc[0], it[0], ang[0], aa[0] = vario["cc1"], vario["it1"], vario["azi1"], vario["hmaj1"]
    anis[0] = vario["hmin1"] / vario["hmaj1"]

    # Assign secondary structure if present
    if nst == 2:
        cc[1], it[1], ang[1], aa[1] = vario["cc2"], vario["it2"], vario["azi2"], vario["hmaj2"]
        anis[1] = vario["hmin2"] / vario["hmaj2"]

    # Compute spatial offsets
    xoff = math.sin(DEG2RAD * azm) * xlag
    yoff = math.cos(DEG2RAD * azm) * xlag
    #print(f"x, y offsets = {xoff:.4f}, {yoff:.4f}")

    # Setup rotation matrix
    rotmat, maxcov = geostats.setup_rotmat(c0, nst, it, cc, ang, 99999.9)

    # Compute variogram model
    xx, yy = 0.0, 0.0  
    for il in range(nlag + 1):
        index[il] = il
        cov[il] = cova2_struct(0.0, 0.0, xx, yy, nst, c0, 9999.9, cc, aa, it, ang, anis, rotmat, maxcov, istruct)

        # Compute semivariance
        gam[il] = (c0 - cov[il]) if istruct == -1 else (cc[istruct] - cov[il])
        
        # Compute correlation
        ro[il] = cov[il] / maxcov
        h[il] = math.sqrt(max(xx**2 + yy**2, 0.0))

        # Update spatial coordinates
        xx += xoff
        yy += yoff

    #print(f"Semivariance values: {gam}")

    return index, h, gam, cov, ro

## **2. `cova2_struct(x1, y1, x2, y2, nst, c0, pmx, cc, aa, it, ang, anis, rotmat, maxcov, istruct)`**

### **Purpose**
Computes **covariance for a given lag distance** based on the **variogram model type** (spherical, exponential, Gaussian, power-law).

### **Inputs**
- `x1, y1, x2, y2`: Coordinates of two spatial points.
- `nst`: Number of nested structures.
- `c0`: Nugget effect.
- `pmx`: Maximum covariance value.
- `cc, aa, it, ang, anis`: Variogram structure parameters.
- `rotmat`: Rotation matrix (for anisotropy).
- `maxcov`: Maximum covariance.
- `istruct`: Index of the variogram structure.

### **Process**
1. **Check for Small Distances**:
   - If **distance is near zero**, returns **covariance equal to the sill**.

2. **Compute Rotated Spatial Distances**:
   - Applies **rotation matrix** to compute anisotropic distances.

3. **Apply Variogram Model**:
   - **Spherical Model**:
     
     $ C(h) = c_i \left( 1 - hr \times (1.5 - 0.5 hr^2) \right), \quad hr < 1 $
     
   - **Exponential Model**:
     
     $ C(h) = c_i \exp\left(-rac{3h}{a_i} \right) $
     
   - **Gaussian Model**:
     
     $ C(h) = c_i \exp\left(-rac{3h^2}{a_i^2} \right) $
     
   - **Power Model**:
     
     $ C(h) = pmx - c_i \times h^{a_i} $

4. **Returns**:
   - Covariance value at given lag distance.

In [ ]:

import math

def cova2_struct(
    x1: float, y1: float, x2: float, y2: float,
    nst: int, c0: float, pmx: float, cc: np.ndarray, aa: np.ndarray,
    it: np.ndarray, ang: np.ndarray, anis: np.ndarray,
    rotmat: np.ndarray, maxcov: float, istruct: int
) -> float:
    """
    Computes the covariance value for a given lag distance based on the specified variogram model.

    Parameters:
    -----------
    x1, y1, x2, y2 : float
        Coordinates of two spatial points.
    nst : int
        Number of nested structures in the variogram model.
    c0 : float
        Nugget effect (discontinuity at the origin).
    pmx : float
        Maximum allowable covariance value.
    cc, aa, it, ang, anis : np.ndarray
        Arrays storing variogram model parameters:
        - `cc`: Partial sill contribution.
        - `aa`: Range parameters.
        - `it`: Model type (1=Spherical, 2=Exponential, 3=Gaussian, 4=Power).
        - `ang`: Variogram azimuth angles.
        - `anis`: Anisotropy ratios (minor/major axis).
    rotmat : np.ndarray
        Rotation matrix for anisotropic transformations.
    maxcov : float
        Maximum covariance value.
    istruct : int
        Index of the variogram structure to compute.

    Returns:
    --------
    float
        Computed covariance value.
    """
    
    EPSILON = 1e-6  # Small threshold to avoid precision issues

    # Compute distance components
    dx, dy = x2 - x1, y2 - y1
    h_sq = dx**2 + dy**2  

    # If distance is near zero, return partial sill
    if h_sq < EPSILON:
        return cc[istruct]

    # Initialize covariance value
    cova2 = 0.0

    # Transform coordinates using rotation matrix (for anisotropic effects)
    dx1 = dx * rotmat[0, istruct] + dy * rotmat[1, istruct]
    dy1 = (dx * rotmat[2, istruct] + dy * rotmat[3, istruct]) / anis[istruct]
    h = math.sqrt(max(dx1**2 + dy1**2, 0.0))  # Ensure h is non-negative

    # Return zero covariance if invalid structure index
    if istruct == -1:
        return 0.0

    # Select and compute the appropriate variogram model
    model_type = it[istruct]

    if model_type == 1:  # Spherical Model
        hr = h / aa[istruct]
        if hr < 1.0:
            cova2 += cc[istruct] * (1.0 - hr * (1.5 - 0.5 * hr**2))

    elif model_type == 2:  # Exponential Model
        cova2 += cc[istruct] * np.exp(-3.0 * h / aa[istruct])

    elif model_type == 3:  # Gaussian Model
        hh = -3.0 * (h**2) / (aa[istruct]**2)
        cova2 += cc[istruct] * np.exp(hh)

    elif model_type == 4:  # Power Model
        cova2 += pmx - cc[istruct] * (h ** aa[istruct])

    return cova2

## **3. `point_pos(x0, y0, d, theta)`**

### **Purpose**
Computes **the spatial coordinates of a point** given an initial location `(x0, y0)`, a distance `d`, and an **angle `theta` (azimuth direction)**.

### **Inputs**
- `x0, y0`: Initial coordinates.
- `d`: Distance to new point.
- `theta`: Azimuth angle (in degrees).

### **Process**
1. Converts **azimuth from degrees to radians**.
2. Computes **new x and y coordinates** using trigonometry:
   
   $ x = x_0 + d \cos(	\theta_{{rad}}) $
   
   $ y = y_0 + d \sin(	\theta_{{rad}}) $

3. **Returns**:
   - `(x, y)`: New point location.

In [ ]:
import math

def point_pos(x0: float, y0: float, d: float, theta: float) -> tuple:
    """
    Computes the new coordinates of a point given an initial position, 
    a distance, and an azimuth angle.

    Parameters:
    -----------
    x0, y0 : float
        Initial coordinates.
    d : float
        Distance to the new point.
    theta : float
        Azimuth angle (in degrees, measured clockwise from north).

    Returns:
    --------
    tuple (float, float)
        The new (x, y) coordinates after translation.
    """
    
    # Convert azimuth angle to radians (measured counterclockwise from x-axis)
    theta_rad = math.radians(90 - theta)

    # Compute new coordinates
    x_new = x0 + d * math.cos(theta_rad)
    y_new = y0 + d * math.sin(theta_rad)

    return x_new, y_new

#### Loading Tabular Data

Here's the command to load our comma delimited data file in to a Pandas' DataFrame object. 

In [ ]:
data = 3

if data == 1:
    df = pd.read_csv("sample_data_MV_biased.csv")
    df = df.rename(columns = {'Por':'Porosity'})            # rename feature(s)
    df['Porosity'] = df['Porosity']*100.0
    df = df.iloc[:,1:]                                      # remove first column
elif data == 2:
    df = pd.read_csv("spatial_nonlinear_MV_facies_v3.csv")
    df = df.rename(columns = {'Por':'Porosity'})            # rename feature(s)
    df = df.iloc[:,1:] 
elif data == 3:
    df = pd.read_csv("12_sample_data.csv")
    df = df.rename(columns = {'Por':'Porosity'})            # rename feature(s) 
    df['Porosity'] = df['Porosity']*100.0
    df = df.iloc[:,1:] 
elif data == 4:
    df = pd.read_csv("spatial_nonlinear_MV_facies_v5_sand_only.csv")
    df = df.rename(columns = {'Por':'Porosity'})            # rename feature(s) 
    df = df.iloc[:,1:] 
else:
    df = pd.read_csv("spatial_nonlinear_MV_facies_v1.csv")
    df = df.rename(columns = {'Por':'Porosity'})            # rename feature(s)
    df = df.iloc[:,1:] 
    
df.head()                                               # we could also use this command for a table preview 

The features:

* **X** - x coordinate in meters
* **Y** - y coordinate in meters
* **Porosity** - rock porosity averaged over a specific rock unit from a vertical well
* **Perm** - rock permeability averaged (scaled up) over a specific rock unit from a vertical well 
* **AI** - acoustic impedance from a seismic cube assigned at a specific rock unit and at the location of a vertical well 
* **facies** - facies, 0 - shale, 1 - sandstone

Concerning facies:

We will work with all facies pooled together. I wanted to simplify this workflow and focus more on spatial continuity direction detection. Finally, by not using facies we do have more samples to support our statistical inference. Most often facies are essential in the subsurface model. Don't worry we will check if this is reasonable in a bit.   

You are welcome to repeat this workflow on a by-facies basis.  The following code could be used to build DataFrames ('df_sand' and 'df_shale') for each facies.

```p
df_sand = pd.DataFrame.copy(df[df['Facies'] == 1]).reset_index()  # copy only 'Facies' = sand records
df_shale = pd.DataFrame.copy(df[df['Facies'] == 0]).reset_index() # copy only 'Facies' = shale records
```

Let's look at summary statistics for all facies combined:

In [ ]:
df.describe().transpose()                               # summary table of sand only DataFrame statistics

#### Set the Model Parameters

See the the following model parameters:

* **xmin**, **xmax**, **ymin** and **ymax** - extents of the dataset for plotting
* **feature** and **feature_units** - feature of interest and associated units
* **vmin** and **vmax** - minimum and maximum of the feature of interest

In [ ]:
xmin = 0.0; xmax = 1000.0                                # spatial extents in x and y
ymin = 0.0; ymax = 1000.0
feature = 'Porosity'; feature_units = 'percentage'         # name and units of the feature of interest
vmin = 0.0; vmax = 25.0                                  # min and max of the feature of interest
cmap = plt.cm.inferno                                    # set the color map

Let's transform the feature to standard normal (mean = 0.0, standard deviation = 1.0, Gaussian shape). This is required for sequential Gaussian simulation (common target for our variogram models) and the Gaussian transform assists with outliers and provides more interpretable variograms. 

Let's look at the inputs for the GeostatsPy nscore program.  Note the output include an ndarray with the transformed values (in the same order as the input data in Dataframe 'df' and column 'vcol'), and the transformation table in original values and also in normal score values. 

The following command will transform the Porosity and Permeabilty to standard normal. 

In [ ]:
#Transform to Gaussian by Facies
df['N' + feature], tvPor, tnsPor = geostats.nscore(df, feature) # nscore transform for all facies porosity 

Let's look at the updated DataFrame to make sure that we now have the normal score porosity and permeability.

In [ ]:
df.head()                                               # preview sand DataFrame with nscore transforms

That looks good! One way to check is to see if the relative magnitudes of the normal score transformed values match the original values.  e.g. that the normal score transform of 0.10 porosity normal score is less than the normal score transform of 0.14 porsity.  Also, the normal score transform of values close to the mean value should be close to 0.0 

Let's also check the original and transformed sand and shale porosity distributions.

In [ ]:
plt.subplot(121)                                        # plot original sand and shale porosity histograms
plt.hist(df[feature], facecolor='red',bins=np.linspace(vmin,vmax,1000),histtype="stepfilled",alpha=0.2,density=True,cumulative=True,edgecolor='black')
plt.xlim([vmin,vmax]); plt.ylim([0,1.0])
plt.xlabel(feature + '(' + feature_units + ')'); plt.ylabel('Frequency'); plt.title('Porosity')
plt.grid(True)

plt.subplot(122)  
plt.hist(df['N'+feature], facecolor='blue',bins=np.linspace(-3.0,3.0,1000),histtype="stepfilled",alpha=0.2,density=True,cumulative=True,edgecolor='black')
plt.xlim([-3.0,3.0]); plt.ylim([0,1.0])
plt.xlabel('Gaussian Transformed ' + feature); plt.ylabel('Frequency'); plt.title('Guassian Transformed ' + feature)
plt.grid(True)

plt.subplots_adjust(left=0.0, bottom=0.0, right=2.0, top=1.2, wspace=0.2, hspace=0.3)
plt.show()

We can see that the normal score transform has correctly transformed the feature to standard normal.

#### Inspection of Posted Data

Data visualization is very useful to detect patterns. Our brains are very good at pattern detection. I promote quantitative methods and recognize issues with cognitive bias, but it is important to recognize the value is expert intepretation based on data visualization.

* This data visualization will also be important to assist with parameter selection for the variogram calculation search template.

Let's plot the location maps of the original feature and the normal score transforms of the feature.

In [ ]:

def locmap_st(df, x_col, y_col, feature_col, xmin, xmax, ymin, ymax, 
              vmin, vmax, title, xlabel, ylabel, cmap="inferno"):
    """
    Plots a spatial location map of a feature using a scatter plot in the current subplot.

    Parameters:
    -----------
    df : pd.DataFrame
        Dataframe containing spatial data.
    x_col, y_col : str
        Column names for X and Y coordinates.
    feature_col : str
        Column name for the feature to be visualized.
    xmin, xmax : float
        Minimum and maximum values for the X-axis.
    ymin, ymax : float
        Minimum and maximum values for the Y-axis.
    vmin, vmax : float
        Minimum and maximum values for the color scale.
    title : str
        Title of the plot.
    xlabel, ylabel : str
        Labels for the X and Y axes.
    cmap : str (default="inferno")
        Colormap for visualization.

    Returns:
    --------
    None (displays the plot)
    """

    ax = plt.gca()  # Get the current subplot
    scatter = ax.scatter(
        df[x_col], df[y_col], c=df[feature_col],
        cmap=cmap, edgecolors="black", vmin=vmin, vmax=vmax
    )

    # Add color bar
    cbar = plt.colorbar(scatter, ax=ax, shrink=0.8)
    cbar.set_label(feature_col)

    # Set axis labels and limits
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_xlim(xmin, xmax)
    ax.set_ylim(ymin, ymax)

    # Set title
    ax.set_title(title)

In [ ]:
plt.figure(figsize=(12, 6))  # Ensure good layout

plt.subplot(121)  # First subplot (Porosity)
locmap_st(df, 'X', 'Y', feature, 0, 1000, 0, 1000, vmin, vmax, 'Porosity', 'X (m)', 'Y (m)', cmap="inferno")

plt.subplot(122)  # Second subplot (Gaussian Transformed Porosity)
locmap_st(df, 'X', 'Y', 'N' + feature, 0, 1000, 0, 1000, -3, 3, 'Gaussian Transformed ' + feature, 'X (m)', 'Y (m)', cmap="inferno")

plt.subplots_adjust(left=0.05, bottom=0.1, right=0.95, top=0.9, wspace=0.3, hspace=0.3)
plt.show()

What do you see?  Here's my observations:

* there is a high degree of spatial agreement between porosity and permeability, this is supported by the high correlation evident in the cross plot.
* there are no discontinuities that could suggest that facies represent a distinct change, rather the porosity and permeability seem continuous and the assigned facies are a truncation of their continous behavoir, we doing 'ok' with no facies
* suspect a 045 azimuth major direction of continuity (up - right)
* there may be cycles in the 135 azimuth 
* there will not likely be a nugget effect, but there is an hint of some short scale discontinuity?

**Do you agree?** If you have a different observations, drop me a line at mpyrcz@austin.utexas.edu and I'll add to this lesson with credit!

#### Experimental Variograms

We can use the location maps to help determine good variogram calculation parameters. For example:

```p
tmin = -9999.; tmax = 9999.; 
lag_dist = 100.0; lag_tol = 50.0; nlag = 7; bandh = 9999.9; azi = azi; atol = 22.5; isill = 1
```
* **tmin**, **tmax** are trimming limits - set to have no impact, no need to filter the data
* **lag_dist**, **lag_tol** are the lag distance, lag tolerance - set based on the common data spacing (100m) and tolerance as 100% of lag distance for additonal smoothing
* **nlag** is number of lags - set to extend just past 50 of the data extent
* **bandh** is the horizontal band width - set to have no effect
* **azi** is the azimuth -  it has not effect since we set atol, the azimuth tolerance, to 90.0
* **isill** is a boolean to standardize the distribution to a variance of 1 - it has no effect since the previous nscore transform sets the variance to 1.0

#### Dashboard for Interactive Variogram Calculation and Modeling

Below we make a dashboard with the ipywidgets and matplotlib Python packages for calculating and modeling experimental variograms.

* allowing you to calculate and model the variogram of the normal score transformed variogram interactively while changing (and exploring) the search template parameters.

* first calculate the isotropic or directional variogram(s)

* then fit the same isotropic or directional variogram(s)

## Need a variogram calculation function

In [ ]:
def make_variogram(nug, nst, it1, c1, ang1, hmaj1, hmin1, it2=None, c2=None, ang2=None, hmaj2=None, hmin2=None):
    """
    Creates a variogram model structure for use in variogram simulations.

    Parameters:
    -----------
    nug : float
        Nugget effect.
    nst : int
        Number of nested structures (1 or 2).
    it1 : int
        Variogram type for first structure (1=Spherical, 2=Exponential, 3=Gaussian, 4=Power).
    c1 : float
        Sill contribution of first structure.
    ang1 : float
        Major azimuth angle for first structure.
    hmaj1 : float
        Major range for first structure.
    hmin1 : float
        Minor range for first structure.
    it2 : int, optional
        Variogram type for second structure (if applicable).
    c2 : float, optional
        Sill contribution of second structure.
    ang2 : float, optional
        Major azimuth angle for second structure.
    hmaj2 : float, optional
        Major range for second structure.
    hmin2 : float, optional
        Minor range for second structure.

    Returns:
    --------
    dict
        Variogram model dictionary.
    """

    vario = {
        "nug": nug,
        "nst": nst,
        "it1": it1,
        "cc1": c1,
        "azi1": ang1,
        "hmaj1": hmaj1,
        "hmin1": hmin1,
    }

    if nst == 2:
        vario.update({
            "it2": it2,
            "cc2": c2,
            "azi2": ang2,
            "hmaj2": hmaj2,
            "hmin2": hmin2,
        })

    return vario

In [ ]:
# interactive calculation of the experimental variogram
l = widgets.Text(value='                              Variogram Calculation Interactive Demonstration, Michael Pyrcz, Associate Professor, The University of Texas at Austin',layout=Layout(width='950px', height='30px'))
lag = widgets.FloatSlider(min = 10, max = 500, value = 10, step = 10, description = 'lag',orientation='vertical',layout=Layout(width='90px', height='200px'))
lag.style.handle_color = 'gray'

lag_tol = widgets.FloatSlider(min = 5, max = 500, value = 5, step = 10, description = 'lag tolerance',orientation='vertical',layout=Layout(width='90px', height='200px'))
lag_tol.style.handle_color = 'gray'

nlag = widgets.IntSlider(min = 1, max = 100, value = 100, step = 1, description = 'number of lags',orientation='vertical',layout=Layout(width='90px', height='200px'))
nlag.style.handle_color = 'gray'

azi = widgets.FloatSlider(min = 0, max = 360, value = 0, step = 5, description = 'azimuth',orientation='vertical',layout=Layout(width='90px', height='200px'))
azi.style.handle_color = 'gray'

azi_tol = widgets.FloatSlider(min = 10, max = 90, value = 10, step = 5, description = 'azimuth tolerance',orientation='vertical',layout=Layout(width='120px', height='200px'))
azi_tol.style.handle_color = 'gray'

bandwidth = widgets.FloatSlider(min = 100, max = 2000, value = 2000, step = 100, description = 'bandwidth',orientation='vertical',layout=Layout(width='90px', height='200px'))
azi_tol.style.handle_color = 'gray'


ui1 = widgets.HBox([lag,lag_tol,nlag,azi,azi_tol,bandwidth],) # basic widget formatting    
ui = widgets.VBox([l,ui1],)

def f_make(lag,lag_tol,nlag,azi,azi_tol,bandwidth):     # function to take parameters, calculate variogram and plot
#    text_trap = io.StringIO()
#    sys.stdout = text_trap
    global lags,gammas,npps,lags2,gammas2,npps2
    tmin = -9999.9; tmax = 9999.9
    lags, gammas, npps = geostats.gamv(df,"X","Y","N"+feature,tmin,tmax,lag,lag_tol,nlag,azi,azi_tol,bandwidth,isill=1.0)
    lags2, gammas2, npps2 = geostats.gamv(df,"X","Y","N"+feature,tmin,tmax,lag,lag_tol,nlag,azi+90.0,azi_tol,bandwidth,isill=1.0)
    
    plt.subplot(111)                                    # plot experimental variogram
    plt.scatter(lags,gammas,color = 'black',s = npps*0.03,label = 'Major Azimuth ' +str(azi), alpha = 0.8)
    plt.scatter(lags2,gammas2,color = 'red',s = npps*0.03,label = 'Minor Azimuth ' +str(azi+90.0), alpha = 0.8)
    plt.plot([0,2000],[1.0,1.0],color = 'black')
    plt.xlabel(r'Lag Distance $\bf(h)$, (m)')
    plt.ylabel(r'$\gamma \bf(h)$')
    if azi_tol < 90.0:
        plt.title('Directional NSCORE ' + feature + ' Variogram - Azi. ' + str(np.round(azi,0)) + ', Azi. Tol.' + str(azi_tol))
    else: 
        plt.title('Omni Directional NSCORE ' + feature + ' Variogram ')
    plt.xlim([0,1000]); plt.ylim([0,1.8])
    plt.legend(loc="lower right")
    plt.grid(True)
    
    plt.subplots_adjust(left=0.0, bottom=0.0, right=1.5, top=1.0, wspace=0.3, hspace=0.3)
    plt.show()
    
    return
    
# connect the function to make the samples and plot to the widgets    
interactive_plot = widgets.interactive_output(f_make, {'lag':lag,'lag_tol':lag_tol,'nlag':nlag,'azi':azi,'azi_tol':azi_tol,'bandwidth':bandwidth})
interactive_plot.clear_output(wait = True)               # reduce flickering by delaying plot updating

### Interactive Variogram Calculation Demonstration

* calculate omnidirectional and direction experimental variograms 

Calculate interpretable experimental variograms for sparse, irregularly-space spatial data. Note, size of the experimental point is scaled by the number of pairs.

* **azimuth** is the azimuth of the lag vector

* **azimuth tolerance** is the maximum allowable departure from the azimuth

* **unit lag distance** the size of the bins in lag distance

* **lag distance tolerance** - the allowable tolerance in lage distance

* **number of lags** - number of lags in the experimental variogram

* **bandwidth** - maximum departure from the lag vector

In [ ]:
display(ui, interactive_plot) 

### Interactive Nested Variogram Modeling Demostration - Nested Structures and Azimuth Interpolation

* select the nested structures and their types, contributions and major and minor ranges, visualize structures and azimuth interpolated variogram 

### The Problem

Fit a positive definite variogram model based on the addition of multiple structures each describing spatial components of the feature variance 

* **nug**: nugget effect

* **c1 / c2**: contributions of the sill - note, **c1** is set at 1.0 - **nug** - **c2**

* **hmaj1 / hmaj2**: range in the major direction

* **hmin1 / hmin2**: range in the minor direction

In [ ]:
# interactive calculation of the sample set (control of source parametric distribution and number of samples)
l = widgets.Text(value='Variogram Modeling, Visualize Nested Structures and Azimuth Interpolation',layout=Layout(width='950px', height='30px'))
#nug = widgets.FloatSlider(min = 0.0001, max = 1.0, value = 0.0001, step = 0.1, description = r'$c_{nugget}$',orientation='vertical',layout=Layout(width='60px', height='200px'))

#Nugget
nug = widgets.FloatSlider(
    min=0.0001, 
    max=1.0, 
    value=0.0001, 
    step=0.1, 
    description=r'$c_{nugget}$',
    orientation='vertical',
    layout=Layout(width='60px', height='220px')  # a bit taller so the text fits
)
nug.style = {'handle_color': 'gray', 'description_position': 'top'}

#nug.style.handle_color = 'gray'
it1 = widgets.Dropdown(options=['Spherical', 'Exponential', 'Gaussian'],value='Spherical',
    description=r'$Type_1$:',disabled=False,layout=Layout(width='200px', height='30px'))
# c1 = widgets.FloatSlider(min=0.0001, max = 1.0, value = 0.2, description = r'$c_1$',orientation='vertical',layout=Layout(width='60px', height='200px'))
# c1.style.handle_color = 'gray'

#hmaj1 = widgets.FloatSlider(min=0.01, max = 10000.0, value = 800.0, step = 25.0, description = r'$a_{1,maj}$',orientation='vertical',layout=Layout(width='60px', height='200px'))
#hmaj1.style.handle_color = 'black'

#hmaj1
hmaj1 = widgets.FloatSlider(
    min=0.01, max=10000, value=800, step=25,
    description=r'$c_{hmaj1}$',
    orientation='vertical',
    layout=Layout(width='60px', height='220px')
)
hmaj1.style = {'handle_color': 'gray', 'description_position': 'top'}


#hmin1 = widgets.FloatSlider(min = 0.01, max = 10000.0, value = 325.0, step = 25.0, description = r'$a_{1,min}$',orientation='vertical',layout=Layout(width='60px', height='200px'))
#hmin1.style.handle_color = 'red'

#hmin1
hmin1 = widgets.FloatSlider(
    min=0.01, 
    max=10000.0, 
    value=325, 
    step=25, 
    description=r'$c_{hmin1}$',
    orientation='vertical',
    layout=Layout(width='60px', height='220px')  # a bit taller so the text fits
)
hmin1.style = {'handle_color': 'gray', 'description_position': 'top'}

it2 = widgets.Dropdown(options=['Spherical', 'Exponential', 'Gaussian'],value='Spherical',
    description=r'$Type_2$:',disabled=False,layout=Layout(width='200px', height='30px'))


#c2 = widgets.FloatSlider(min=0.0001, max = 1.0, value = 0.0001, description = r'$c_2$',orientation='vertical',layout=Layout(width='60px', height='200px'))
#c2.style.handle_color = 'gray'
#c2
c2 = widgets.FloatSlider(
    min=0.0001, 
    max=1.0, 
    value=0.0001, 
    step=0.1, 
    description=r'$c_{c2}$',
    orientation='vertical',
    layout=Layout(width='60px', height='220px')  # a bit taller so the text fits
)
c2.style = {'handle_color': 'gray', 'description_position': 'top'}

#hmaj2 = widgets.FloatSlider(min=0.01, max = 10000.0, value = 800.0, step = 25.0, description = r'$a_{2,maj}$',orientation='vertical',layout=Layout(width='60px', height='200px'))
#hmaj2.style.handle_color = 'black'
#hmaj2
hmaj2 = widgets.FloatSlider(
    min=0.01, 
    max=10000.0, 
    value=800, 
    step=25, 
    description=r'$c_{hmaj2}$',
    orientation='vertical',
    layout=Layout(width='60px', height='220px')  # a bit taller so the text fits
)
hmaj2.style = {'handle_color': 'gray', 'description_position': 'top'}

#hmin2 = widgets.FloatSlider(min = 0.01, max = 10000.0, value = 325.0, step = 25.0, description = r'$a_{2,min}$',orientation='vertical',layout=Layout(width='60px', height='200px'))
#hmin2.style.handle_color = 'red'
#hmin2
hmin2 = widgets.FloatSlider(
    min=0.01, 
    max=10000.0, 
    value=325, 
    step=25, 
    description=r'$c_{hmin2}$',
    orientation='vertical',
    layout=Layout(width='60px', height='220px')  # a bit taller so the text fits
)
hmin2.style = {'handle_color': 'gray', 'description_position': 'top'}

#new_azimuth = widgets.FloatSlider(min = 0.0, max = 360.0, value = 45.0, step = 5.0, description = r'$azi_{inter}$',orientation='vertical',layout=Layout(width='60px', height='200px'))
#new_azimuth.style.handle_color = 'purple'
#new_azimuth
new_azimuth = widgets.FloatSlider(
    min=0.0, 
    max=360, 
    value=45, 
    step=5, 
    description=r'$c_{new_azimuth}$',
    orientation='vertical',
    layout=Layout(width='60px', height='220px')  # a bit taller so the text fits
)
new_azimuth.style = {'handle_color': 'gray', 'description_position': 'top'}

ui9 = widgets.HBox([nug,it1,hmaj1,hmin1,it2,c2,hmaj2,hmin2,new_azimuth],)                   # basic widget formatting   
#ui2 = widgets.HBox([it2,c2,hmaj2,hmin2],)                   # basic widget formatting   
ui10 = widgets.VBox([l,ui9],)

def convert_type(it):
    if it == 'Spherical': 
        return 1
    elif it == 'Exponential':
        return 2
    else: 
        return 3

def f_make2(nug,it1,hmaj1,hmin1,it2,c2,hmaj2,hmin2,new_azimuth):                       # function to take parameters, make sample and plot
    azimuth = azi.value
    c1 = 1.0 - nug - c2
    it1 = convert_type(it1); it2 = convert_type(it2)
    if c2 > 0.0:
        nst = 2
    else:
        nst = 1
    
    vario = make_variogram(nug,nst,it1,c1,0.0,hmaj1,hmin1,it2,c2,0.0,hmaj2,hmin2) # make model object
    nlag = 100; xlag = 10;                                     
    index_maj,h_maj,gam_maj,cov_maj,ro_maj = geostats.vmodel(nlag,xlag,0.0,vario)   # project the model in the major azimuth                                                  # project the model in the 135 azimuth
    index_min,h_min,gam_min,cov_min,ro_min = geostats.vmodel(nlag,xlag,90.0,vario)
    index_new,h_new,gam_new,cov_new,ro_new = geostats.vmodel(nlag,xlag,azimuth-new_azimuth,vario)
    
    _,h_maj0,gam_maj0,_,_ = vmodel_struct(nlag,xlag,0.0,vario,-1) 
    _,h_maj1,gam_maj1,_,_ = vmodel_struct(nlag,xlag,0.0,vario,0) 
    _,h_maj2,gam_maj2,_,_ = vmodel_struct(nlag,xlag,0.0,vario,1) 
    _,h_min0,gam_min0,_,_ = vmodel_struct(nlag,xlag,90.0,vario,-1) 
    _,h_min1,gam_min1,_,_ = vmodel_struct(nlag,xlag,90.0,vario,0) 
    _,h_min2,gam_min2,_,_ = vmodel_struct(nlag,xlag,90.0,vario,1) 
    _,h_new0,gam_new0,_,_ = vmodel_struct(nlag,xlag,azimuth-new_azimuth,vario,-1) 
    _,h_new1,gam_new1,_,_ = vmodel_struct(nlag,xlag,azimuth-new_azimuth,vario,0) 
    _,h_new2,gam_new2,_,_ = vmodel_struct(nlag,xlag,azimuth-new_azimuth,vario,1) 
    
    plt.subplot(221)                                    # plot experimental variogram
    plt.scatter(lags,gammas,color = 'black',s = npps*0.03,label = 'Major Azimuth ' +str(azimuth), alpha = 0.8,zorder=10)
    plt.plot(h_maj,gam_maj,color = 'black',lw=3,zorder=10)
    if nug > 0.0001: 
        plt.plot(h_maj0,gam_maj0,color = 'black',lw=1.5)
        plt.fill_between(h_maj,gam_maj0,np.full(len(h_maj),0),color='grey',alpha=1.0,zorder=1,label = 'Nugget')
    if c1 > 0.0001: 
        plt.plot(h_maj,gam_maj1+gam_maj0,color = 'black',lw=1.5)
        plt.fill_between(h_maj,gam_maj1+gam_maj0,gam_maj0,color='darkorange',alpha=1.0,zorder=1,label = 'Structure #1')
    
    if c2 > 0.0001:   
        plt.plot(h_maj,gam_maj2+gam_maj1+gam_maj0,color = 'black',lw=1.5)
        plt.fill_between(h_maj,gam_maj2+gam_maj1+gam_maj0,gam_maj1+gam_maj0,color='deepskyblue',alpha=1.0,zorder=1,label='Structure #2')
    
    plt.plot([0,2000],[1.0,1.0],color = 'black',ls='--')
    plt.xlabel(r'Lag Distance $\bf(h)$, (m)'); plt.ylabel(r'$\gamma \bf(h)$')
    if azi_tol.value < 90.0:
        plt.title('Major Directional NSCORE ' + feature + ' Variogram - Azi. ' + str(azimuth))
    else: 
        plt.title('Omni Directional NSCORE ' + feature + ' Variogram ')

    if c1 > 0.0001:
        plt.vlines(hmaj1,0,1.8,color='black',lw=1.5); 
        plt.annotate('Structure 1 Range',[hmaj1-30,1.3],rotation=90.0);
    if c2 > 0.0001:
        plt.vlines(hmaj2,0,1.8,color='black',lw=2.0)
        plt.annotate('Structure 2 Range',[hmaj2-30,1.3],color='black',rotation=90.0)
    plt.xlim([0,1000]); plt.ylim([0,1.8])
    plt.legend(loc="upper left")
    plt.grid(True)
    
    plt.subplot(222)                                    # plot experimental variogram
    plt.scatter(lags2,gammas2,color = 'red',s = npps*0.03,label = 'Minor Azimuth ' +str(azimuth+90.0), alpha = 0.8,zorder=10)
    plt.plot(h_min,gam_min,color = 'red',lw=3)
    if nug > 0.0001:
        plt.plot(h_min0,gam_min0,color = 'red',lw=1.5)
        plt.fill_between(h_min,gam_min0,np.full(len(h_maj),0),color='grey',alpha=1.0,zorder=1,label = 'Nugget')
    if c1 > 0.0001:  
        plt.plot(h_min,gam_min1+gam_min0,color = 'red',lw=1.5)
        plt.fill_between(h_min,gam_min1+gam_min0,gam_min0,color='darkorange',alpha=1.0,zorder=1,label = 'Structure #1')
    
    if c2 > 0.0001:   
        plt.plot(h_min,gam_min2+gam_min1+gam_min0,color = 'red',lw=1.5)
        plt.fill_between(h_min,gam_min2+gam_min1+gam_min0,gam_min1+gam_min0,color='deepskyblue',alpha=1.0,zorder=1,label='Structure #2')
    
    plt.plot([0,2000],[1.0,1.0],color = 'black',ls='--')
    plt.xlabel(r'Lag Distance $\bf(h)$, (m)')
    plt.ylabel(r'$\gamma \bf(h)$')
    if azi_tol.value < 90.0:
        plt.title('Minor Directional NSCORE ' + feature + ' Variogram - Azi. ' + str(azimuth + 90.0))
    else: 
        plt.title('Omni Directional NSCORE ' + feature + ' Variogram ')
    if c1 > 0.0001:      
        plt.vlines(hmin1,0,1.8,color='red',lw=1.5)
        plt.annotate('Structure 1 Range',[hmin1-30,1.3],color='red',rotation=90.0)
    if c2 > 0.0001:  
        plt.vlines(hmin2,0,1.8,color='red',lw=2.0)
        plt.annotate('Structure 2 Range',[hmin2-30,1.3],color='red',rotation=90.0)
    plt.xlim([0,1000]); plt.ylim([0,1.8])
    plt.legend(loc="upper left")
    plt.grid(True)
    
    plt.subplot(223)                                    # plot experimental variogram
    plt.plot(h_new,gam_new,color = 'purple',lw=3)
    if nug > 0.0001:
        plt.plot(h_new0,gam_new0,color = 'purple',lw=1.5)
        plt.fill_between(h_new,gam_new0,np.full(len(h_maj),0),color='grey',alpha=1.0,zorder=1,label = 'Nugget')
    if c1 > 0.0001:
        plt.plot(h_new,gam_new1+gam_new0,color = 'purple',lw=1.5)
        plt.fill_between(h_new,gam_new1+gam_new0,gam_new0,color='darkorange',alpha=1.0,zorder=1,label = 'Structure #1')
    
    if c2 > 0.0001:   
        plt.plot(h_new,gam_new2+gam_new1+gam_new0,color = 'purple',lw=1.5)
        plt.fill_between(h_new,gam_new2+gam_new1+gam_new0,gam_new1+gam_new0,color='deepskyblue',alpha=1.0,zorder=1,label='Structure #2')
    
    plt.plot([0,2000],[1.0,1.0],color = 'black',ls='--')
    plt.xlabel(r'Lag Distance $\bf(h)$, (m)')
    plt.ylabel(r'$\gamma \bf(h)$')
    if azi_tol.value < 90.0:
        plt.title('Interpolated ' + feature + ' Variogram - Azi. ' + str(new_azimuth))
    else: 
        plt.title('Omni Directional NSCORE ' + feature + ' Variogram ')
        
    if c1 > 0.0001:
        plt.vlines(hmaj1,0,1.8,color='black',lw=1.5); 

    if c1 > 0.0001:      
        plt.vlines(hmin1,0,1.8,color='red',lw=1.5)
        
    if c2 > 0.0001:
        plt.vlines(hmaj2,0,1.8,color='black',lw=1.5); 
  
    if c2 > 0.0001:      
        plt.vlines(hmin2,0,1.8,color='red',lw=1.5)
    
    # plt.vlines(hmin1,0,1.8,color='black',lw=1.5); plt.vlines(hmin2,0,1.8,color='deepskyblue',lw=1.5)
    # plt.annotate('Structure 1 Range',[hmin1-30,1.3],rotation=90.0); plt.annotate('Structure 2 Range',[hmin2-30,1.3],color='deepskyblue',rotation=90.0)
    plt.xlim([0,1000]); plt.ylim([0,1.8])
    plt.legend(loc="upper left")
    plt.grid(True)
    
    plt.subplot(224)
    
    plt.xlim([-2000,2000]); plt.ylim([-2000,2000]); plt.xlabel('X offset (m)'); plt.ylabel('Y offset (m)')
    plt.plot([-2000,2000],[0,0],color='grey',lw=3,zorder=1); plt.plot([0,0],[-2000,2000],color='grey',lw=3,zorder=1)
    plt.grid(True); plt.title('2D Variogram Structures - Ranges and Geometric Anisotropy')
    
    if c1 > 0.0001:
        e1 = patches.Ellipse((0, 0), hmaj1*2, hmin1*2,angle=90-azimuth, linewidth=2, fill=True,facecolor='darkorange',edgecolor='black',alpha=1.0,zorder=4)
        plt.gca().add_patch(e1)
    
    if c2 > 0.0001:
        e2 = patches.Ellipse((0, 0), hmaj2*2, hmin2*2,angle=90-azimuth, linewidth=2, fill=True,facecolor='deepskyblue',edgecolor='black',alpha=1.0,zorder=2)
        plt.gca().add_patch(e2)
    
    xarr,yarr = point_pos(0, 0, hmaj1, azimuth)
    plt.plot([0,xarr],[0,yarr],color='black',zorder=13)
    
    xarr2,yarr2 = point_pos(0, 0, hmaj2, azimuth)
    plt.plot([0,xarr2],[0,yarr2],color='black',zorder=10)
    plt.annotate('Major',[xarr2,yarr2],zorder=10)
    
    xarr1,yarr1 = point_pos(0, 0, hmin1, azimuth+90.0)
    plt.plot([0,xarr1],[0,yarr1],color='red',zorder=13)
    
    xarr2,yarr2 = point_pos(0, 0, hmin2, azimuth+90.0)
    plt.plot([0,xarr2],[0,yarr2],color='red',zorder=10)
    plt.annotate('Minor',[xarr2,yarr2],zorder=10,color='red')
    
    xarr_new,yarr_new = point_pos(0, 0, 1200, new_azimuth)
    plt.plot([0,xarr_new],[0,yarr_new],color='purple',zorder=10,lw=3)
    plt.annotate('Interpolated Azimuth',[xarr_new,yarr_new],color='purple',zorder=10)
    
    plt.subplots_adjust(left=0.0, bottom=0.0, right=2.0, top=2.5, wspace=0.2, hspace=0.2)
    plt.show()
    
# connect the function to make the samples and plot to the widgets    
interactive_plot2 = widgets.interactive_output(f_make2, {'nug':nug, 'it1':it1, 'hmaj1':hmaj1, 'hmin1':hmin1, 'it2':it2, 'c2':c2, 'hmaj2':hmaj2, 'hmin2':hmin2,'new_azimuth':new_azimuth})
interactive_plot2.clear_output(wait = True)               # reduce flickering by delaying plot updating  

In [ ]:
display(ui10, interactive_plot2)  